In [ ]:
import torch as T
from torch import nn as nn
from torch.nn import functional as F
from embedding import CharacterConvEmbedding
from attention import BasicAttention, BiAttention, CoAttention
from network import EmbedText, Encoder, DynamicPointingDecoder

In [ ]:
char_args = {'vocab_size': 70, 'embedding_dim': 8, 'padding_idx':10}
char_conv_args = {'char_embed_dim':8, 'embedding_dim':100, 'kernel_sizes':[2, 3, 4 ,5], 'num_filters':50}
word_args = {'vocab_size':40000, 'embedding_dim':100, 'padding_idx':10}

c_char = T.zeros([32, 600, 50], dtype=T.long)
c_word = T.zeros([32, 600], dtype=T.long)
q_char = T.zeros([32, 50, 50], dtype=T.long)
q_word = T.zeros([32, 50], dtype=T.long)

In [ ]:
embedtext = EmbedText(word_args, char_args, shared=False)
qw, cw, qc, cc = embedtext(q_word, c_word, q_char, c_char)
qw = qw.cuda()
cw = cw.cuda()
qc = qc.cuda()
cc = cc.cuda()

print (qw.type(), cw.type(), qc.type(), cc.type())

In [ ]:
encoder = Encoder(word_args, char_conv_args, hidden_dim=200, shared=True, use_cuda=True)

q_lstm, c_lstm = encoder(qw, cw, qc, cc)

print (q_lstm.type(), c_lstm.type())
print (q_lstm.shape, c_lstm.shape)

In [ ]:
i = CoAttention(200, use_cuda=True)
o = i(q_lstm, c_lstm)

o.shape

In [ ]:
dn = DynamicPointingDecoder(hidden_dim=400, use_cuda=True)

In [ ]:
dn(o)